In [1]:
!pip install tensorboard

In [2]:
import yaml
from lstm import *
from mlp import *
from gensim.models import Word2Vec
import pandas as pd

In [3]:
# Verificar se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
fi = open('config.yaml', 'r')
card_dict = yaml.load(fi, Loader=yaml.FullLoader)

In [5]:
def indice_para_nome(indice, cards_dict):
    """Converte um índice para o nome da carta correspondente usando o dicionário de cartas."""
    try:
        return cards_dict['cards_list'][indice]
    except IndexError:
        raise ValueError(f"Índice {indice} fora do alcance do dicionário de cartas.")


In [6]:
df_train = pd.read_csv('train_df.csv')
df_train = df_train.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
#df_train = df_train[:90]

In [7]:
word_to_vec_model = Word2Vec.load("card2vec.model")

In [8]:
from dataset_class import *

# Crie o dataset e o DataLoader
dataset = MagicDataset(df_train, word_to_vec_model, card_dict)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# Itere sobre as primeiras 45 entradas do DataLoader
for i, (inputs, labels, mascara) in enumerate(dataloader):
    if i < 45:
        print(f"--- Entrada {i+1} ---")
        
        # Converta os embeddings para nomes
        input_names = [MagicDataset.embedding_para_nome(embedding.numpy(), word_to_vec_model) for embedding in inputs.squeeze(0)]
        
        # Converta o índice da label para o nome usando a função indice_para_nome
        label_name = indice_para_nome(labels.item(), card_dict)
        print(labels.item())
        print("Inputs:", input_names)
        print("Label:", label_name)
        print("Máscara:", mascara)
        print("\n")
        break
    else:
        break


--- Entrada 1 ---
70
Inputs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Label: Skyrider_Patrol
Máscara: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [9]:
# Instanciar o modelo LSTM
from torch import nn
#model = LSTMModel()
model = FeedForwardModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
import os
import torch
from torch.utils.tensorboard import SummaryWriter
from dataset_class import *
from torch.utils.data import DataLoader

# Defina os hiperparâmetros
num_epochs = 300
batch_size = 8

# Configurar o TensorBoard SummaryWriter para escrever logs na pasta 'runs'
log_dir = './runs'
writer = SummaryWriter(log_dir=log_dir)

# Defina um diretório para salvar os checkpoints
checkpoint_dir = 'checkpoints'

# Certifique-se de que o diretório exista (ou crie-o)
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Criar o dataset e o DataLoader
dataset = MagicDataset(df_train, word_to_vec_model, card_dict)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Definir seu modelo, otimizador e função de perda aqui
# Certifique-se de inicializar 'model' e 'optimizer' corretamente

# Mover o modelo para a GPU
model.to(device)

# Defina uma variável para rastrear o número total de iterações
total_iterations = 0

# Ciclo de treinamento
for epoch in range(num_epochs):
    total_loss = 0.0
    total_batches = 0

    for i, (inputs, labels, mascara) in enumerate(dataloader):
        # Mover as entradas e rótulos para a GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        mascara = mascara.to(device)

        # Zerar os gradientes
        optimizer.zero_grad()
        # Forward
        outputs = model(inputs, mascara)
        loss = criterion(outputs, labels)

        # Acumular a perda
        total_loss += loss.item()
        total_batches += 1

        # Backward
        loss.backward()
        optimizer.step()

        # Atualizar o número total de iterações
        total_iterations += 1

        # Salvar um checkpoint a cada determinado número de iterações (por exemplo, a cada 1000 iterações)
        if total_iterations % 100000 == 0:
            checkpoint = {
                'epoch': epoch,
                'total_iterations': total_iterations,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss.item()
            }
            checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_{total_iterations}.pt')
            torch.save(checkpoint, checkpoint_path)

    # Calcular a perda média da época e registrar no TensorBoard
    average_loss = total_loss / total_batches
    writer.add_scalar('Epoch Loss', average_loss, epoch)

    # Imprimir a perda média da época
    print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss:.4f}")

# Fechar o writer após terminar o treinamento
writer.close()


/home/diego/Documentos/MTG_drafter/draftsym/dataset_class.py:66: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  X = torch.tensor(pool_embedding, dtype=torch.float32)


KeyboardInterrupt: 

# Teste

In [ ]:
import yaml
from lstm import *
from mlp import *
from gensim.models import Word2Vec
import pandas as pd

df_test = pd.read_csv('test_df.csv')
df_test = df_test.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
#df_train = df_train[:200]

In [ ]:
fi = open('config.yaml', 'r')
card_dict = yaml.load(fi, Loader=yaml.FullLoader)

In [ ]:
import os
import torch
from dataset_class import *
from torch.utils.data import DataLoader

# Carregar o modelo e otimizador do checkpoint
def load_model_from_checkpoint(model, optimizer, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    total_iterations = checkpoint['total_iterations']
    loss = checkpoint['loss']
    return model, optimizer, epoch, total_iterations, loss

# Configurações
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint_path = 'checkpoints/checkpoint_best.pt'  # Aqui, assumindo que você deseja carregar o "melhor" checkpoint

# Instanciar modelo e otimizador
model = FeedForwardModel()  # Substitua pelo nome correto da sua classe de modelo
optimizer = None  # Você não precisa do otimizador durante o teste, mas é útil para carregar o modelo a partir do checkpoint

# Carregar o modelo do checkpoint
model, _, _, _, _ = load_model_from_checkpoint(model, optimizer, checkpoint_path)

# Mover modelo para o dispositivo apropriado
model.to(device)
model.eval()

# Criar o dataset e o DataLoader para teste
dataset_test = MagicDataset(df_test, word_to_vec_model, card_dict)  # Aqui, assumindo que 'df_test' é o seu dataframe de teste
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

# Variáveis para rastrear a precisão
total_samples = 0
correct_samples = 0

# Ciclo de teste
with torch.no_grad():
    for i, (inputs, labels, mascara) in enumerate(test_loader):
        # Mover entradas e rótulos para o dispositivo
        inputs = inputs.to(device)
        labels = labels.to(device)
        mascara = mascara.to(device)
        
        # Predição
        outputs = model(inputs, mascara)
        _, predicted = torch.max(outputs, 1)
        
        # Atualizar contadores de precisão
        total_samples += labels.size(0)
        correct_samples += (predicted == labels).sum().item()

        # Opcional: Imprimir estatísticas a cada 100 lotes
        if i % 100 == 0:
            print(f"Batch [{i+1}/{len(test_loader)}] - Accuracy: {100 * correct_samples / total_samples:.2f}%")

# Imprimir precisão final
print(f"Total Accuracy: {100 * correct_samples / total_samples:.2f}%")


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/checkpoint_best.pt'